In [4]:
import importlib
import numpy as np

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import notebookutils as nbutl

# Classification Results

In [5]:
league_standing_pytable = MyPyTable()
league_standing_pytable.load_from_file("lib/nhl_leaguestandings.csv")

league_standing_pytable.remove_rows_with_missing_values()

league_standing_pytable.drop_col("ROW")
league_standing_pytable.drop_col("RPt%")
league_standing_pytable.drop_col("OL")
league_standing_pytable.drop_col("T")

#nbutl.discretize_ptspercent(league_standing_pytable.data, )

key = league_standing_pytable.get_key(league_standing_pytable.column_names, ['PTS%'])[0]
nbutl.discretize_ptspercent(league_standing_pytable.data, key)

y_col_name = "PLAYOFFS"

y = league_standing_pytable.get_column(y_col_name)
league_standing_pytable.drop_col(y_col_name)
X = [league_standing_pytable.data[i][6:] for i in range(len(league_standing_pytable.data))]

print(X)

X_train_folds, X_test_folds  = myevaluation.stratified_kfold_cross_validation(X, y, 10)


[[82.0, 47.0, 24.0, 105.0, '7', 248.0, 196.0, 0.62, -0.02], [82.0, 37.0, 26.0, 93.0, '6', 231.0, 196.0, 0.43, 0.0], [82.0, 38.0, 30.0, 90.0, '6', 242.0, 225.0, 0.21, 0.0], [82.0, 33.0, 38.0, 77.0, '5', 217.0, 227.0, -0.08, 0.04], [82.0, 24.0, 48.0, 58.0, '4', 194.0, 244.0, -0.55, 0.06], [82.0, 44.0, 23.0, 103.0, '7', 239.0, 179.0, 0.73, 0.0], [82.0, 45.0, 30.0, 97.0, '6', 268.0, 231.0, 0.46, 0.01], [82.0, 39.0, 30.0, 91.0, '6', 214.0, 181.0, 0.4, 0.0], [82.0, 37.0, 28.0, 91.0, '6', 207.0, 175.0, 0.41, 0.02], [82.0, 32.0, 39.0, 75.0, '5', 184.0, 209.0, -0.26, 0.05], [82.0, 34.0, 30.0, 86.0, '6', 210.0, 202.0, 0.1, 0.0], [82.0, 30.0, 34.0, 78.0, '5', 210.0, 228.0, -0.21, 0.01], [82.0, 31.0, 45.0, 68.0, '5', 200.0, 218.0, -0.21, 0.01], [82.0, 19.0, 54.0, 47.0, '3', 179.0, 292.0, -1.28, 0.09], [82.0, 43.0, 32.0, 93.0, '6', 245.0, 202.0, 0.46, -0.06], [82.0, 37.0, 32.0, 87.0, '6', 237.0, 209.0, 0.29, -0.05], [82.0, 29.0, 41.0, 70.0, '5', 202.0, 248.0, -0.55, 0.01], [82.0, 28.0, 47.0, 63.0, 

In [6]:
# descision tree

descision_tree = MyDecisionTreeClassifier()
descision_tree.fit(X, y)

descision_tree.print_decision_rules(league_standing_pytable.column_names[6:], y_col_name)
descision_tree.visualize_tree(dot_fname="out/tree.dot", pdf_fname="out/tree.pdf", attribute_names=league_standing_pytable.column_names[6:])

IF SRS=-3.09 THEN PLAYOFFS=N
IF SRS=-2.47 THEN PLAYOFFS=N
IF SRS=-2.29 THEN PLAYOFFS=N
IF SRS=-2.22 THEN PLAYOFFS=N
IF SRS=-2.1 THEN PLAYOFFS=N
IF SRS=-1.96 THEN PLAYOFFS=N
IF SRS=-1.94 THEN PLAYOFFS=N
IF SRS=-1.91 THEN PLAYOFFS=N
IF SRS=-1.86 THEN PLAYOFFS=N
IF SRS=-1.83 THEN PLAYOFFS=N
IF SRS=-1.77 THEN PLAYOFFS=N
IF SRS=-1.75 THEN PLAYOFFS=N
IF SRS=-1.72 THEN PLAYOFFS=N
IF SRS=-1.71 THEN PLAYOFFS=N
IF SRS=-1.61 THEN PLAYOFFS=N
IF SRS=-1.57 THEN PLAYOFFS=N
IF SRS=-1.56 THEN PLAYOFFS=N
IF SRS=-1.55 THEN PLAYOFFS=N
IF SRS=-1.54 THEN PLAYOFFS=N
IF SRS=-1.53 THEN PLAYOFFS=N
IF SRS=-1.45 THEN PLAYOFFS=N
IF SRS=-1.44 THEN PLAYOFFS=N
IF SRS=-1.43 THEN PLAYOFFS=Y
IF SRS=-1.42 THEN PLAYOFFS=N
IF SRS=-1.38 THEN PLAYOFFS=N
IF SRS=-1.37 THEN PLAYOFFS=N
IF SRS=-1.36 THEN PLAYOFFS=N
IF SRS=-1.34 THEN PLAYOFFS=N
IF SRS=-1.33 THEN PLAYOFFS=N
IF SRS=-1.32 THEN PLAYOFFS=N
IF SRS=-1.31 THEN PLAYOFFS=N
IF SRS=-1.3 THEN PLAYOFFS=N
IF SRS=-1.29 THEN PLAYOFFS=N
IF SRS=-1.28 THEN PLAYOFFS=N
IF SRS=-1.27 THE

In [7]:
random_forest = MyRandomForestClassifier()


X_train, X_test, y_train, y_test = myevaluation.train_test_split(X, y)
random_forest.fit([X_train], [X_test], N=10, M=5, F=2)

predictions = random_forest.predict(y_train)

print(predictions)


904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904


TypeError: 'int' object is not iterable